In [2]:
from pymongo import MongoClient
from googletrans import Translator
from bson.objectid import ObjectId
import tqdm
import re
from nltk import ngrams
import Levenshtein
import pickle
import os
from gensim.test.utils import lee_corpus_list
from gensim.models import Word2Vec, Doc2Vec
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


In [3]:
client = MongoClient(host='103.252.1.144', username='admin', password = 'CIST2o20')
cb = client['human_resources']['career_builder']
cv = cb.find({'eng_experience':{'$exists':True}, 'fulltext':{'$regex':'.*developer.*'}})
jpt = client['human_resources']['job_posting_topcv']
jd = jpt.find({'eng_job_description':{'$regex':'.*developer.*'}})

In [4]:
class CSO_classifier():
    def __init__(self):
        path = './ontology/ontology/'
        self.skill_ontology = {}
        for file in os.listdir(path):
            if file.endswith(".p"):
                with open(os.path.join(path, file), 'rb') as f:
                    try:
                        while True:
                            self.skill_ontology[file.replace('_ontology.p','')]=pickle.load(f)
                    except EOFError:
                        pass
        self.doc2vec = Doc2Vec()

    def concept_similarity(self, sentence):
        sentence = re.sub(r"[-()\"/@;:<>{}`=~|.!?,]", " ", sentence).lower()

        uni_gram = ngrams(sentence.split(), 1)
        bi_gram = ngrams(sentence.split(), 2)
        tri_gram = ngrams(sentence.split(), 3)
        uni_gram = [i[0] for i in list(uni_gram)]
        bi_gram = ['{} {}'.format(i[0],i[1]) for i in list(bi_gram)]
        tri_gram = ['{} {} {}'.format(i[0],i[1], i[2]) for i in list(tri_gram)]
        n_gram = uni_gram + bi_gram + tri_gram

        skill_from_cv_or_jd = []

        for i in n_gram:
            for o_k in self.skill_ontology.keys():
                for k in self.skill_ontology[o_k]['topics_wu'].values():
                    if Levenshtein.ratio(i, str(k)) > 0.94:
                        if k in self.skill_ontology[o_k]['broaders']:
                            skill = {'rank_0': o_k, 'rank_1': self.skill_ontology[o_k]['broaders'][k], 'rank_2': str(k)}
                            skill_from_cv_or_jd.append(skill)



        soft_skill = [i for i in skill_from_cv_or_jd if i['rank_0']=='softskill']
        general_skill = [i for i in skill_from_cv_or_jd if i['rank_0']=='general']
        other_skill = [i for i in skill_from_cv_or_jd if (i['rank_0']!='softskill' and i['rank_0']!='general')]

        return (general_skill, other_skill), soft_skill

    def semantic_similarity(self,sentence):
        pass

In [5]:
ontology = {'A':[], 'JD': [], 'R': []}

c = {'Profile': None, 'Work_history': None, 'Education': None, 'Course': None, 'Skills': {}, 'Pos': None, 'Others': None}
d = {'Information': None, 'Descriptions': None, 'Required': None, 'Skills': {}}

cso_classifier = CSO_classifier()

c['Skills']['Technical_skills'], c['Skills']['Soft_skills'] = cso_classifier.concept_similarity(' '.join(cv[0]['eng_experience']+cv[0]['eng_skills']))
d['Skills']['Technical_skills'], d['Skills']['Soft_skills'] = cso_classifier.concept_similarity(jd[0]['title']+' '+' '.join(jd[0]['eng_job_description']+jd[0]['eng_requirement']))